### Reference Dates

Num Training Weeks = 4  
Num Test/Val Days = 15

- Train: 5/30/2017 (Tues), 6/6/2017,6/13/2017,6/20/2017  
- Val: 7/11/2017 (Tues) - 7/25/2017 (Wed)  
- Test: 8/1/2017 (Tues)  - 8/15/2017 (Wed)  

### Import packages

In [26]:
from datetime import date, timedelta, datetime
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from tqdm import tqdm

### Import training data

In [27]:
tic = datetime.now()
df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)
toc = datetime.now()

In [3]:
print(toc - tic)

0:03:34.408020


In [30]:
np.expm1(df_train.unit_sales).max()

89440.00000000003

### Import items and stores data

In [4]:
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../input/stores.csv",
).set_index("store_nbr")

In [5]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [6]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Filter the data to 2017 and after

In [7]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

In [8]:
gc.collect()

0

In [9]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


### Import and modify promotion data

In [10]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train, df_test
gc.collect()

21

In [11]:
promo_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 243 columns]

### Transform sales training data

In [12]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [13]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [14]:
df_2017_index = df_2017.index

### Transform items data

In [15]:
items['class'] = items['class'].astype('category')
items = pd.get_dummies(items)

In [16]:
items.head()

,perishable,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,...,class_6920,class_6922,class_6924,class_6936,class_6954,class_6960,class_7002,class_7016,class_7034,class_7780
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103501,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
103520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103665,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
items = items.reindex(df_2017.index.get_level_values(1))
items.shape

(167515, 371)

### Transform Stores data

In [18]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [19]:
stores['cluster'] = stores.cluster.astype('category')
stores = pd.get_dummies(stores)

In [20]:
stores = stores.reindex(df_2017.index.get_level_values(0))
stores.shape

(167515, 60)

In [21]:
stores.head()

,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,...,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17
store_nbr,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
#sum of sales for each item across all stores for each day
#df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
#df_2017_item.shape

In [23]:
#sum of promotion for each item across all stores for each day
#promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

In [24]:
# sum of sales in each class (multiple items) for each store for each day 
# df_2017_store_class = df_2017.reset_index()
# df_2017_store_class['class'] = items['class'].values
# df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
# df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

In [25]:
#df_2017_store_class_index.head()

In [26]:
#df_2017_store_class.shape

In [27]:
#df_2017_store_class.head()

In [28]:
# sum of promotions in each class (multiple items) for each store for each day 

# df_2017_promo_store_class = promo_2017.reset_index()
# df_2017_promo_store_class['class'] = items['class'].values
# df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
# df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [29]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [30]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_7_2017": get_timespan(promo_df, t2017, 7, 7).sum(axis=1).values,
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_30_2017": get_timespan(promo_df, t2017, 30, 30).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30]:
        tmp1 = get_timespan(df, t2017, i, i)
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
                
        
    for i in [3, 7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
    X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 15):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
    for i in range(-14, 15):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=15)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

### Prepare training data

In [31]:
print("Preparing dataset...")
t2017 = date(2017, 5, 30)
num_days = 4
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp = pd.concat([X_tmp, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

Preparing dataset...


100%|██████████| 4/4 [01:06<00:00, 16.58s/it]


14

### Prepare validation and test data

In [32]:
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 11))

# X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
# X_val2.index = df_2017_item.index
# X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

# X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
# X_val3.index = df_2017_store_class.index
# X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 1), is_train=False)

# X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
# X_test2.index = df_2017_item.index
# X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

# X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
# X_test3.index = df_2017_store_class.index
# X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

# del X_test2, X_val2, df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
# gc.collect()

In [33]:
#Check shapes to ensure consistency
X_train.shape, y_train.shape

((670060, 547), (670060, 15))

In [34]:
X_val.shape, y_val.shape

((167515, 547), (167515, 15))

In [35]:
X_test.shape

(167515, 547)

In [36]:
del df_2017, promo_2017
gc.collect()

0

In [31]:
import itertools

In [32]:
#create prarmeter grid
feature_fraction_grid = [0.8,0.9,1]
bagging_fraction_grid = [0.6,0.8,1]
num_leaves_grid = [31, 200] #31 is the default

param_grid = list(itertools.product(num_leaves_grid, feature_fraction_grid, bagging_fraction_grid))

In [33]:
param_grid

[(31, 0.8, 0.6),
 (31, 0.8, 0.8),
 (31, 0.8, 1),
 (31, 0.9, 0.6),
 (31, 0.9, 0.8),
 (31, 0.9, 1),
 (31, 1, 0.6),
 (31, 1, 0.8),
 (31, 1, 1),
 (200, 0.8, 0.6),
 (200, 0.8, 0.8),
 (200, 0.8, 1),
 (200, 0.9, 0.6),
 (200, 0.9, 0.8),
 (200, 0.9, 1),
 (200, 1, 0.6),
 (200, 1, 0.8),
 (200, 1, 1)]

## Run #1 (Run in seperate cells to prevent total loss in crash)

In [40]:
val_results_run_1 = []


start_value = 0
end_value = 6

for num, grid_item in enumerate(param_grid[start_value:end_value],0): 

              
    params = {
        'num_leaves': grid_item[0],
        'objective': 'regression',
#        'min_data_in_leaf': 200,
        'learning_rate': 0.02,
        'feature_fraction': grid_item[1],
        'bagging_fraction': grid_item[2],
        'bagging_freq': 1,
        'metric': 'l2',
        'num_threads': 0
    }

    print("*" * 50)
    print(f'Model {num}')
    print(f'num_leaves: {params["num_leaves"]}')
    print(f'feature_fraction: {params["feature_fraction"]}')
    print(f'bagging_fraction: {params["bagging_fraction"]}')
    print("*" * 50)
    
    MAX_ROUNDS = 5000
    val_pred = []
    test_pred = []
    cate_vars = []
    feature_importance = []
    
    for i in range(15):
        tic = datetime.now()
        print("-" * 50)
        print("Step %d" % (i+1))
        print("-" * 50)
        dtrain = lgb.Dataset(
            X_train, label=y_train[:, i],
            categorical_feature=cate_vars,
            weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
        )
        dval = lgb.Dataset(
            X_val, label=y_val[:, i], reference=dtrain,
            weight=items["perishable"] * 0.25 + 1,
            categorical_feature=cate_vars)
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
        )
#         print("\n".join(("%s: %.2f" % x) for x in sorted(
#             zip(X_train.columns, bst.feature_importance("gain")),
#             key=lambda x: x[1], reverse=True
#         )))
        
        feature_importance.append(bst.feature_importance("gain"))
              
        #predict with validation data
        val_pred.append(bst.predict(
            X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

        #predict with test data
        test_pred.append(bst.predict(
            X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
        toc = datetime.now()
        print(f'model runtime: {toc-tic}')

    #calculate MSE
    print("Validation mse:", mean_squared_error(
        y_val, np.array(val_pred).transpose()))
    
    #calculate validation error
    weight = items["perishable"] * 0.25 + 1
    err = (y_val - np.array(val_pred).transpose())**2
    err = err.sum(axis=1) * weight
    err = np.sqrt(err.sum() / weight.sum() / 15)
    print('nwrmsle = {}'.format(err))
    val_results_run_1.append(err)

    folder_path = '../derived_datasets/'
    #save data
    df_preds_test = pd.DataFrame(
        np.array(test_pred).transpose(), index=df_2017_index,
        columns=pd.date_range("2017-08-01", periods=15)
    )
    df_preds_test.index.set_names(["store_nbr", "item_nbr"], inplace=True)
    df_preds_test.to_csv(folder_path + 'lgbm_test_pred_model_' + str(num) +'.csv')
    
    #save featue_importance
    df_feature_importance = pd.DataFrame(np.array(feature_importance).transpose(),
                                         index = X_train.columns,
                                        columns = pd.date_range('2017-08-01', periods=15))
    df_feature_importance.to_csv(folder_path + 'lgbm_feature_importance_model_' + str(num) +'.csv')

print(val_results_run_1)

**************************************************
Model 0
num_leaves: 31
feature_fraction: 0.8
bagging_fraction: 0.6
**************************************************
--------------------------------------------------
Step 1
--------------------------------------------------


/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.43423	valid_1's l2: 0.417095
[100]	training's l2: 0.33501	valid_1's l2: 0.323948
[150]	training's l2: 0.316773	valid_1's l2: 0.308679
[200]	training's l2: 0.311206	valid_1's l2: 0.304618
[250]	training's l2: 0.308514	valid_1's l2: 0.303108
[300]	training's l2: 0.306623	valid_1's l2: 0.302171
[350]	training's l2: 0.305234	valid_1's l2: 0.301655
[400]	training's l2: 0.304112	valid_1's l2: 0.301356
[450]	training's l2: 0.303187	valid_1's l2: 0.30117
[500]	training's l2: 0.3023	valid_1's l2: 0.300987
[550]	training's l2: 0.301481	valid_1's l2: 0.300845
[600]	training's l2: 0.300723	valid_1's l2: 0.300729
[650]	training's l2: 0.300018	valid_1's l2: 0.300651
[700]	training's l2: 0.299343	valid_1's l2: 0.300558
[750]	training's l2: 0.298703	valid_1's l2: 0.300482
[800]	training's l2: 0.298062	valid_1's l2: 0.300393
[850]	training's l2: 0.297459	valid_1's l2: 0.300321
[900]	training's l2: 0.296885	valid_1's l2:

[1600]	training's l2: 0.312176	valid_1's l2: 0.325486
[1650]	training's l2: 0.31169	valid_1's l2: 0.32548
[1700]	training's l2: 0.311181	valid_1's l2: 0.32546
[1750]	training's l2: 0.310687	valid_1's l2: 0.32546
[1800]	training's l2: 0.310179	valid_1's l2: 0.32545
[1850]	training's l2: 0.309714	valid_1's l2: 0.325433
[1900]	training's l2: 0.309215	valid_1's l2: 0.325423
[1950]	training's l2: 0.308754	valid_1's l2: 0.325403
[2000]	training's l2: 0.308278	valid_1's l2: 0.325398
[2050]	training's l2: 0.307831	valid_1's l2: 0.325392
[2100]	training's l2: 0.307344	valid_1's l2: 0.325384
[2150]	training's l2: 0.306878	valid_1's l2: 0.325372
[2200]	training's l2: 0.306411	valid_1's l2: 0.325329
[2250]	training's l2: 0.305957	valid_1's l2: 0.325324
[2300]	training's l2: 0.305518	valid_1's l2: 0.325315
[2350]	training's l2: 0.305087	valid_1's l2: 0.325333
[2400]	training's l2: 0.304651	valid_1's l2: 0.325301
[2450]	training's l2: 0.304261	valid_1's l2: 0.325309
[2500]	training's l2: 0.303839	va

[450]	training's l2: 0.336755	valid_1's l2: 0.341975
[500]	training's l2: 0.335713	valid_1's l2: 0.341761
[550]	training's l2: 0.334775	valid_1's l2: 0.341684
[600]	training's l2: 0.333858	valid_1's l2: 0.341565
[650]	training's l2: 0.333013	valid_1's l2: 0.341493
[700]	training's l2: 0.332193	valid_1's l2: 0.341469
[750]	training's l2: 0.331423	valid_1's l2: 0.341382
[800]	training's l2: 0.33071	valid_1's l2: 0.341398
[850]	training's l2: 0.329938	valid_1's l2: 0.341303
[900]	training's l2: 0.329191	valid_1's l2: 0.341292
[950]	training's l2: 0.328509	valid_1's l2: 0.341263
[1000]	training's l2: 0.327819	valid_1's l2: 0.34126
[1050]	training's l2: 0.327159	valid_1's l2: 0.341295
[1100]	training's l2: 0.326555	valid_1's l2: 0.341302
Early stopping, best iteration is:
[986]	training's l2: 0.328014	valid_1's l2: 0.341243
model runtime: 0:01:04.809556
--------------------------------------------------
Step 9
--------------------------------------------------
Training until validation scor

...

/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.434168	valid_1's l2: 0.416924
[100]	training's l2: 0.335039	valid_1's l2: 0.324053
[150]	training's l2: 0.31669	valid_1's l2: 0.308665
[200]	training's l2: 0.311408	valid_1's l2: 0.304899
[250]	training's l2: 0.3086	valid_1's l2: 0.303273
[300]	training's l2: 0.30663	valid_1's l2: 0.302288
[350]	training's l2: 0.305225	valid_1's l2: 0.301744
[400]	training's l2: 0.304124	valid_1's l2: 0.301463
[450]	training's l2: 0.30323	valid_1's l2: 0.301287
[500]	training's l2: 0.302413	valid_1's l2: 0.301101
[550]	training's l2: 0.301705	valid_1's l2: 0.300973
[600]	training's l2: 0.300976	valid_1's l2: 0.300839
[650]	training's l2: 0.300329	valid_1's l2: 0.300758
[700]	training's l2: 0.299756	valid_1's l2: 0.300703
[750]	training's l2: 0.299199	valid_1's l2: 0.300638
[800]	training's l2: 0.298698	valid_1's l2: 0.300587
[850]	training's l2: 0.298213	valid_1's l2: 0.300557
[900]	training's l2: 0.297754	valid_1's l2:

[1100]	training's l2: 0.31912	valid_1's l2: 0.325894
[1150]	training's l2: 0.318589	valid_1's l2: 0.325867
[1200]	training's l2: 0.318082	valid_1's l2: 0.325851
[1250]	training's l2: 0.317594	valid_1's l2: 0.325838
[1300]	training's l2: 0.317094	valid_1's l2: 0.325834
[1350]	training's l2: 0.316617	valid_1's l2: 0.325812
[1400]	training's l2: 0.316148	valid_1's l2: 0.325783
[1450]	training's l2: 0.315673	valid_1's l2: 0.325766
[1500]	training's l2: 0.315215	valid_1's l2: 0.325767
[1550]	training's l2: 0.314799	valid_1's l2: 0.325764
Early stopping, best iteration is:
[1433]	training's l2: 0.315831	valid_1's l2: 0.325751
model runtime: 0:01:35.624514
--------------------------------------------------
Step 5
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.501615	valid_1's l2: 0.47324
[100]	training's l2: 0.386703	valid_1's l2: 0.370731
[150]	training's l2: 0.365622	valid_1's l2: 0.355247
[200]	training

[450]	training's l2: 0.324304	valid_1's l2: 0.331239
[500]	training's l2: 0.32327	valid_1's l2: 0.331038
[550]	training's l2: 0.32234	valid_1's l2: 0.330878
[600]	training's l2: 0.321525	valid_1's l2: 0.330763
[650]	training's l2: 0.320762	valid_1's l2: 0.330671
[700]	training's l2: 0.320016	valid_1's l2: 0.330557
[750]	training's l2: 0.319409	valid_1's l2: 0.330507
[800]	training's l2: 0.318784	valid_1's l2: 0.33047
[850]	training's l2: 0.318231	valid_1's l2: 0.330459
[900]	training's l2: 0.317684	valid_1's l2: 0.330422
[950]	training's l2: 0.317132	valid_1's l2: 0.33037
[1000]	training's l2: 0.316614	valid_1's l2: 0.330345
[1050]	training's l2: 0.316119	valid_1's l2: 0.33032
[1100]	training's l2: 0.315637	valid_1's l2: 0.330288
[1150]	training's l2: 0.315158	valid_1's l2: 0.330274
[1200]	training's l2: 0.314691	valid_1's l2: 0.330259
[1250]	training's l2: 0.31424	valid_1's l2: 0.330242
[1300]	training's l2: 0.313779	valid_1's l2: 0.330237
[1350]	training's l2: 0.313351	valid_1's l2: 

[250]	training's l2: 0.370027	valid_1's l2: 0.388835
[300]	training's l2: 0.367196	valid_1's l2: 0.38795
[350]	training's l2: 0.365367	valid_1's l2: 0.387515
[400]	training's l2: 0.363697	valid_1's l2: 0.386956
[450]	training's l2: 0.362325	valid_1's l2: 0.386785
[500]	training's l2: 0.361192	valid_1's l2: 0.386846
[550]	training's l2: 0.360156	valid_1's l2: 0.386842
[600]	training's l2: 0.359195	valid_1's l2: 0.386697
[650]	training's l2: 0.358312	valid_1's l2: 0.386743
[700]	training's l2: 0.357462	valid_1's l2: 0.386845
Early stopping, best iteration is:
[623]	training's l2: 0.358791	valid_1's l2: 0.386648
model runtime: 0:00:59.804190
--------------------------------------------------
Step 15
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.480667	valid_1's l2: 0.470283
[100]	training's l2: 0.387865	valid_1's l2: 0.387215
[150]	training's l2: 0.369097	valid_1's l2: 0.374041
[200]	training's l2: 0.

### Run #2

In [ ]:
val_results_run_2 = []

start_value = 6
end_value = 12

for num, grid_item in enumerate(param_grid[start_value:end_value],start_value): 

              
    params = {
        'num_leaves': grid_item[0],
        'objective': 'regression',
#        'min_data_in_leaf': 200,
        'learning_rate': 0.02,
        'feature_fraction': grid_item[1],
        'bagging_fraction': grid_item[2],
        'bagging_freq': 1,
        'metric': 'l2',
        'num_threads': 0
    }

    print("*" * 50)
    print(f'Model {num}')
    print(f'num_leaves: {params["num_leaves"]}')
    print(f'feature_fraction: {params["feature_fraction"]}')
    print(f'bagging_fraction: {params["bagging_fraction"]}')
    print("*" * 50)
    
    MAX_ROUNDS = 5000
    val_pred = []
    test_pred = []
    cate_vars = []
    feature_importance = []
    
    for i in range(15):
        tic = datetime.now()
        print("-" * 50)
        print("Step %d" % (i+1))
        print("-" * 50)
        dtrain = lgb.Dataset(
            X_train, label=y_train[:, i],
            categorical_feature=cate_vars,
            weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
        )
        dval = lgb.Dataset(
            X_val, label=y_val[:, i], reference=dtrain,
            weight=items["perishable"] * 0.25 + 1,
            categorical_feature=cate_vars)
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
        )
#         print("\n".join(("%s: %.2f" % x) for x in sorted(
#             zip(X_train.columns, bst.feature_importance("gain")),
#             key=lambda x: x[1], reverse=True
#         )))
        
        feature_importance.append(bst.feature_importance("gain"))
              
        #predict with validation data
        val_pred.append(bst.predict(
            X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

        #predict with test data
        test_pred.append(bst.predict(
            X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
        toc = datetime.now()
        print(f'model runtime: {toc-tic}')

    #calculate MSE
    print("Validation mse:", mean_squared_error(
        y_val, np.array(val_pred).transpose()))
    
    #calculate validation error
    weight = items["perishable"] * 0.25 + 1
    err = (y_val - np.array(val_pred).transpose())**2
    err = err.sum(axis=1) * weight
    err = np.sqrt(err.sum() / weight.sum() / 15)
    print('nwrmsle = {}'.format(err))
    val_results_run_2.append(err)

    folder_path = '../derived_datasets/'
    #save data
    df_preds_test = pd.DataFrame(
        np.array(test_pred).transpose(), index=df_2017_index,
        columns=pd.date_range("2017-08-01", periods=15)
    )
    df_preds_test.index.set_names(["store_nbr", "item_nbr"], inplace=True)
    df_preds_test.to_csv(folder_path + 'lgbm_test_pred_model_' + str(num) +'.csv')
    
    #save featue_importance
    df_feature_importance = pd.DataFrame(np.array(feature_importance).transpose(),
                                         index = X_train.columns,
                                        columns = pd.date_range('2017-08-01', periods=15))
    df_feature_importance.to_csv(folder_path + 'lgbm_feature_importance_model_' + str(num) +'.csv')

print(val_results_run_2)

**************************************************
Model 6
num_leaves: 31
feature_fraction: 1
bagging_fraction: 0.6
**************************************************
--------------------------------------------------
Step 1
--------------------------------------------------


/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.433914	valid_1's l2: 0.416631
[100]	training's l2: 0.334907	valid_1's l2: 0.323782
[150]	training's l2: 0.316627	valid_1's l2: 0.308567
[200]	training's l2: 0.311272	valid_1's l2: 0.304842
[250]	training's l2: 0.308539	valid_1's l2: 0.303322
[300]	training's l2: 0.306611	valid_1's l2: 0.302441
[350]	training's l2: 0.305241	valid_1's l2: 0.301942
[400]	training's l2: 0.304083	valid_1's l2: 0.301616
[450]	training's l2: 0.30315	valid_1's l2: 0.301402
[500]	training's l2: 0.302242	valid_1's l2: 0.301215
[550]	training's l2: 0.301432	valid_1's l2: 0.301118
[600]	training's l2: 0.300633	valid_1's l2: 0.300975
[650]	training's l2: 0.299926	valid_1's l2: 0.3009
[700]	training's l2: 0.299239	valid_1's l2: 0.300853
[750]	training's l2: 0.298576	valid_1's l2: 0.300739
[800]	training's l2: 0.297929	valid_1's l2: 0.300614
[850]	training's l2: 0.297299	valid_1's l2: 0.300521
[900]	training's l2: 0.296704	valid_1's l

[1500]	training's l2: 0.313032	valid_1's l2: 0.325607
[1550]	training's l2: 0.312483	valid_1's l2: 0.325596
[1600]	training's l2: 0.311925	valid_1's l2: 0.325577
[1650]	training's l2: 0.311407	valid_1's l2: 0.325539
[1700]	training's l2: 0.31086	valid_1's l2: 0.325487
[1750]	training's l2: 0.310321	valid_1's l2: 0.325488
[1800]	training's l2: 0.309791	valid_1's l2: 0.325469
[1850]	training's l2: 0.30928	valid_1's l2: 0.325449
[1900]	training's l2: 0.308787	valid_1's l2: 0.325433
[1950]	training's l2: 0.30833	valid_1's l2: 0.325441
[2000]	training's l2: 0.307821	valid_1's l2: 0.325463
Early stopping, best iteration is:
[1922]	training's l2: 0.308594	valid_1's l2: 0.325414
model runtime: 0:01:51.790949
--------------------------------------------------
Step 5
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.501198	valid_1's l2: 0.472696
[100]	training's l2: 0.386462	valid_1's l2: 0.370427
[150]	training

[400]	training's l2: 0.337895	valid_1's l2: 0.34213
[450]	training's l2: 0.336717	valid_1's l2: 0.341898
[500]	training's l2: 0.335629	valid_1's l2: 0.341696
[550]	training's l2: 0.334687	valid_1's l2: 0.341611
[600]	training's l2: 0.333746	valid_1's l2: 0.341489
[650]	training's l2: 0.332893	valid_1's l2: 0.341434
[700]	training's l2: 0.332016	valid_1's l2: 0.341341
[750]	training's l2: 0.331231	valid_1's l2: 0.341267
[800]	training's l2: 0.33051	valid_1's l2: 0.341257
[850]	training's l2: 0.329775	valid_1's l2: 0.341194
[900]	training's l2: 0.329022	valid_1's l2: 0.341141
[950]	training's l2: 0.328338	valid_1's l2: 0.341099
[1000]	training's l2: 0.32764	valid_1's l2: 0.341054
[1050]	training's l2: 0.326958	valid_1's l2: 0.341051
[1100]	training's l2: 0.326328	valid_1's l2: 0.341072
[1150]	training's l2: 0.325699	valid_1's l2: 0.341101
Early stopping, best iteration is:
[1046]	training's l2: 0.327009	valid_1's l2: 0.34104
model runtime: 0:01:12.588978
---------------------------------

...

Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.417707	valid_1's l2: 0.404892
[100]	training's l2: 0.319559	valid_1's l2: 0.316014
[150]	training's l2: 0.300728	valid_1's l2: 0.30302
[200]	training's l2: 0.293721	valid_1's l2: 0.300427
[250]	training's l2: 0.289122	valid_1's l2: 0.299548
[300]	training's l2: 0.285317	valid_1's l2: 0.299119
[350]	training's l2: 0.282068	valid_1's l2: 0.298924
[400]	training's l2: 0.279267	valid_1's l2: 0.298796
[450]	training's l2: 0.276699	valid_1's l2: 0.298703
[500]	training's l2: 0.27433	valid_1's l2: 0.298633
[550]	training's l2: 0.272084	valid_1's l2: 0.298591
[600]	training's l2: 0.269917	valid_1's l2: 0.298573
[650]	training's l2: 0.267827	valid_1's l2: 0.298558
[700]	training's l2: 0.265754	valid_1's l2: 0.298576
Early stopping, best iteration is:
[621]	training's l2: 0.269047	valid_1's l2: 0.298551
model runtime: 0:01:37.375028
--------------------------------------------------
Step 2
-----------------------

[550]	training's l2: 0.290268	valid_1's l2: 0.328655
Early stopping, best iteration is:
[464]	training's l2: 0.294633	valid_1's l2: 0.328522
model runtime: 0:01:24.317113
--------------------------------------------------
Step 10
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.431715	valid_1's l2: 0.455438
[100]	training's l2: 0.348156	valid_1's l2: 0.368584
[150]	training's l2: 0.330577	valid_1's l2: 0.353089
[200]	training's l2: 0.323061	valid_1's l2: 0.349222
[250]	training's l2: 0.317748	valid_1's l2: 0.347651
[300]	training's l2: 0.313412	valid_1's l2: 0.34697
[350]	training's l2: 0.309773	valid_1's l2: 0.346621
[400]	training's l2: 0.306609	valid_1's l2: 0.346546
[450]	training's l2: 0.303814	valid_1's l2: 0.346533
[500]	training's l2: 0.301136	valid_1's l2: 0.34652
[550]	training's l2: 0.298704	valid_1's l2: 0.346535
Early stopping, best iteration is:
[474]	training's l2: 0.302489	valid_1's l2

/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.417669	valid_1's l2: 0.40491
[100]	training's l2: 0.3195	valid_1's l2: 0.316051
[150]	training's l2: 0.300717	valid_1's l2: 0.303071
[200]	training's l2: 0.293708	valid_1's l2: 0.300451
[250]	training's l2: 0.289095	valid_1's l2: 0.29956
[300]	training's l2: 0.285152	valid_1's l2: 0.298995
[350]	training's l2: 0.281881	valid_1's l2: 0.298768
[400]	training's l2: 0.279298	valid_1's l2: 0.298716
[450]	training's l2: 0.277007	valid_1's l2: 0.298745
[500]	training's l2: 0.274883	valid_1's l2: 0.298714
[550]	training's l2: 0.272916	valid_1's l2: 0.298692
[600]	training's l2: 0.271017	valid_1's l2: 0.298706
[650]	training's l2: 0.269227	valid_1's l2: 0.298727
Early stopping, best iteration is:
[541]	training's l2: 0.273267	valid_1's l2: 0.29869
model runtime: 0:01:28.573643
--------------------------------------------------
Step 2
--------------------------------------------------
Training until validation sc

[450]	training's l2: 0.304326	valid_1's l2: 0.346791
Early stopping, best iteration is:
[361]	training's l2: 0.309031	valid_1's l2: 0.346708
model runtime: 0:01:17.353324
--------------------------------------------------
Step 11
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.455118	valid_1's l2: 0.457442
[100]	training's l2: 0.354583	valid_1's l2: 0.364088
[150]	training's l2: 0.333492	valid_1's l2: 0.348646
[200]	training's l2: 0.324744	valid_1's l2: 0.344797
[250]	training's l2: 0.318826	valid_1's l2: 0.343109
[300]	training's l2: 0.314165	valid_1's l2: 0.342196
[350]	training's l2: 0.310134	valid_1's l2: 0.341639
[400]	training's l2: 0.306981	valid_1's l2: 0.341608
[450]	training's l2: 0.304349	valid_1's l2: 0.341626
[500]	training's l2: 0.301878	valid_1's l2: 0.341697
[550]	training's l2: 0.299599	valid_1's l2: 0.341716
Early stopping, best iteration is:
[427]	training's l2: 0.305506	valid_1's 

### Run #3

In [39]:
val_results_run_3 = []

start_value = 12
end_value = 18

for num, grid_item in enumerate(param_grid[start_value:end_value],start_value): 

              
    params = {
        'num_leaves': grid_item[0],
        'objective': 'regression',
#        'min_data_in_leaf': 200,
        'learning_rate': 0.02,
        'feature_fraction': grid_item[1],
        'bagging_fraction': grid_item[2],
        'bagging_freq': 1,
        'metric': 'l2',
        'num_threads': 0
    }

    print("*" * 50)
    print(f'Model {num}')
    print(f'num_leaves: {params["num_leaves"]}')
    print(f'feature_fraction: {params["feature_fraction"]}')
    print(f'bagging_fraction: {params["bagging_fraction"]}')
    print("*" * 50)
    
    MAX_ROUNDS = 5000
    val_pred = []
    test_pred = []
    cate_vars = []
    feature_importance = []
    
    for i in range(15):
        tic = datetime.now()
        print("-" * 50)
        print("Step %d" % (i+1))
        print("-" * 50)
        dtrain = lgb.Dataset(
            X_train, label=y_train[:, i],
            categorical_feature=cate_vars,
            weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
        )
        dval = lgb.Dataset(
            X_val, label=y_val[:, i], reference=dtrain,
            weight=items["perishable"] * 0.25 + 1,
            categorical_feature=cate_vars)
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
        )
#         print("\n".join(("%s: %.2f" % x) for x in sorted(
#             zip(X_train.columns, bst.feature_importance("gain")),
#             key=lambda x: x[1], reverse=True
#         )))
        
        feature_importance.append(bst.feature_importance("gain"))
              
        #predict with validation data
        val_pred.append(bst.predict(
            X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

        #predict with test data
        test_pred.append(bst.predict(
            X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
        toc = datetime.now()
        print(f'model runtime: {toc-tic}')

    #calculate MSE
    print("Validation mse:", mean_squared_error(
        y_val, np.array(val_pred).transpose()))
    
    #calculate validation error
    weight = items["perishable"] * 0.25 + 1
    err = (y_val - np.array(val_pred).transpose())**2
    err = err.sum(axis=1) * weight
    err = np.sqrt(err.sum() / weight.sum() / 15)
    print('nwrmsle = {}'.format(err))
    val_results_run_3.append(err)

    folder_path = '../derived_datasets/'
    #save data
    df_preds_test = pd.DataFrame(
        np.array(test_pred).transpose(), index=df_2017_index,
        columns=pd.date_range("2017-08-01", periods=15)
    )
    df_preds_test.index.set_names(["store_nbr", "item_nbr"], inplace=True)
    df_preds_test.to_csv(folder_path + 'lgbm_test_pred_model_' + str(num) +'.csv')
    
    #save featue_importance
    df_feature_importance = pd.DataFrame(np.array(feature_importance).transpose(),
                                         index = X_train.columns,
                                        columns = pd.date_range('2017-08-01', periods=15))
    df_feature_importance.to_csv(folder_path + 'lgbm_feature_importance_model_' + str(num) +'.csv')

print(val_results_run_3)

**************************************************
Model 12
num_leaves: 200
feature_fraction: 0.9
bagging_fraction: 0.6
**************************************************
--------------------------------------------------
Step 1
--------------------------------------------------


/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.417044	valid_1's l2: 0.404212
[100]	training's l2: 0.319507	valid_1's l2: 0.31605
[150]	training's l2: 0.300696	valid_1's l2: 0.303067
[200]	training's l2: 0.293829	valid_1's l2: 0.300574
[250]	training's l2: 0.289253	valid_1's l2: 0.299861
[300]	training's l2: 0.285433	valid_1's l2: 0.299351
[350]	training's l2: 0.282151	valid_1's l2: 0.299072
[400]	training's l2: 0.279276	valid_1's l2: 0.298947
[450]	training's l2: 0.276696	valid_1's l2: 0.298917
[500]	training's l2: 0.274198	valid_1's l2: 0.298873
[550]	training's l2: 0.271845	valid_1's l2: 0.298872
[600]	training's l2: 0.269586	valid_1's l2: 0.298798
[650]	training's l2: 0.267456	valid_1's l2: 0.298823
[700]	training's l2: 0.265357	valid_1's l2: 0.298823
Early stopping, best iteration is:
[595]	training's l2: 0.269804	valid_1's l2: 0.298788
model runtime: 0:01:33.981696
--------------------------------------------------
Step 2
----------------------

model runtime: 0:01:20.431280
--------------------------------------------------
Step 10
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.431198	valid_1's l2: 0.45485
[100]	training's l2: 0.34815	valid_1's l2: 0.368523
[150]	training's l2: 0.330627	valid_1's l2: 0.353121
[200]	training's l2: 0.323156	valid_1's l2: 0.349464
[250]	training's l2: 0.317857	valid_1's l2: 0.347969
[300]	training's l2: 0.313472	valid_1's l2: 0.347207
[350]	training's l2: 0.309843	valid_1's l2: 0.346919
[400]	training's l2: 0.306596	valid_1's l2: 0.346734
[450]	training's l2: 0.303718	valid_1's l2: 0.346757
[500]	training's l2: 0.301011	valid_1's l2: 0.346748
[550]	training's l2: 0.298389	valid_1's l2: 0.346796
[600]	training's l2: 0.295913	valid_1's l2: 0.346773
Early stopping, best iteration is:
[485]	training's l2: 0.301806	valid_1's l2: 0.346722
model runtime: 0:01:21.580819
-----------------------------------------------

/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


...

Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.416779	valid_1's l2: 0.403921
[100]	training's l2: 0.319283	valid_1's l2: 0.315747
[150]	training's l2: 0.300571	valid_1's l2: 0.303076
[200]	training's l2: 0.293558	valid_1's l2: 0.300583
[250]	training's l2: 0.288817	valid_1's l2: 0.299677
[300]	training's l2: 0.284877	valid_1's l2: 0.299264
[350]	training's l2: 0.281612	valid_1's l2: 0.299073
[400]	training's l2: 0.278792	valid_1's l2: 0.298935
[450]	training's l2: 0.276167	valid_1's l2: 0.298861
[500]	training's l2: 0.273725	valid_1's l2: 0.298814
[550]	training's l2: 0.271462	valid_1's l2: 0.298806
[600]	training's l2: 0.269252	valid_1's l2: 0.298815
[650]	training's l2: 0.267133	valid_1's l2: 0.298791
Early stopping, best iteration is:
[535]	training's l2: 0.272106	valid_1's l2: 0.298775
model runtime: 0:01:38.952891
--------------------------------------------------
Step 2
--------------------------------------------------
Training until validati

model runtime: 0:01:38.964917
--------------------------------------------------
Step 10
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.430782	valid_1's l2: 0.454536
[100]	training's l2: 0.347868	valid_1's l2: 0.368372
[150]	training's l2: 0.330258	valid_1's l2: 0.353047
[200]	training's l2: 0.322672	valid_1's l2: 0.349293
[250]	training's l2: 0.317338	valid_1's l2: 0.34779
[300]	training's l2: 0.312895	valid_1's l2: 0.347125
[350]	training's l2: 0.309185	valid_1's l2: 0.346868
[400]	training's l2: 0.30604	valid_1's l2: 0.346745
[450]	training's l2: 0.303243	valid_1's l2: 0.346754
[500]	training's l2: 0.300585	valid_1's l2: 0.346725
[550]	training's l2: 0.298113	valid_1's l2: 0.346769
[600]	training's l2: 0.295694	valid_1's l2: 0.346782
Early stopping, best iteration is:
[495]	training's l2: 0.300833	valid_1's l2: 0.346716
model runtime: 0:01:37.356043
-----------------------------------------------

/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.416817	valid_1's l2: 0.403807
[100]	training's l2: 0.319458	valid_1's l2: 0.315779
[150]	training's l2: 0.300753	valid_1's l2: 0.303084
[200]	training's l2: 0.293732	valid_1's l2: 0.300623
[250]	training's l2: 0.289019	valid_1's l2: 0.299732
[300]	training's l2: 0.284995	valid_1's l2: 0.299198
[350]	training's l2: 0.281722	valid_1's l2: 0.299024
[400]	training's l2: 0.279106	valid_1's l2: 0.298974
[450]	training's l2: 0.276866	valid_1's l2: 0.298937
[500]	training's l2: 0.274784	valid_1's l2: 0.298899
[550]	training's l2: 0.272792	valid_1's l2: 0.298902
[600]	training's l2: 0.270895	valid_1's l2: 0.298929
Early stopping, best iteration is:
[512]	training's l2: 0.274287	valid_1's l2: 0.298883
model runtime: 0:01:38.733100
--------------------------------------------------
Step 2
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training'

Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.454305	valid_1's l2: 0.456701
[100]	training's l2: 0.354344	valid_1's l2: 0.364208
[150]	training's l2: 0.333168	valid_1's l2: 0.348736
[200]	training's l2: 0.324391	valid_1's l2: 0.345064
[250]	training's l2: 0.318611	valid_1's l2: 0.343572
[300]	training's l2: 0.313992	valid_1's l2: 0.342687
[350]	training's l2: 0.309948	valid_1's l2: 0.342125
[400]	training's l2: 0.306758	valid_1's l2: 0.342083
[450]	training's l2: 0.30403	valid_1's l2: 0.342104
[500]	training's l2: 0.30169	valid_1's l2: 0.342159
Early stopping, best iteration is:
[405]	training's l2: 0.306444	valid_1's l2: 0.342072
model runtime: 0:01:33.897298
--------------------------------------------------
Step 12
--------------------------------------------------
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.502275	valid_1's l2: 0.478859
[100]	training's l2: 0.387301	valid_1's l2: 0.38807
[150]	training's 

In [23]:
#Manually compiling the result due to computer crash
val_result_run_1 = [0.5872718050674727, 0.5873143138903639, 0.5876759499194352, 0.5874139858966546, 0.5874023160917615, 0.5878217115658666]
val_result_run_2 = [0.5875244205187689, 0.5874967800573019, 0.5878745768757881, 0.5863037771886317, 0.5861946713091547, 0.5863312204831337]
val_result_run_3 = [0.5864005897634857, 0.5863076932051245, 0.5865439595121598, 0.5865449715362678, 0.5864578642166351, 0.5867543150265673]

In [24]:
val_result_all  = val_result_run_1 + val_result_run_2 + val_result_run_3

In [ ]:
import numpy as np

In [25]:
lowest_index = np.argmin(vaL_result_all)
print(f'model_{lowest_index} has the lowest validation error')
print(f'nwrmsle is {val_result_all[lowest_index]}')

model_10 has the lowest validation error
nwrmsle is 0.5861946713091547


model_10 has the lowest validation error
